In [2]:
import numpy as np
from pprint import pprint
from scipy.optimize import linprog

In [3]:
def check_lpsolver(A_mat, b_vec):
    m, n = A_mat.shape
    c_vec = np.random.rand(n)
    c_vec = np.zeros(n)
    sol = linprog(c_vec, -A_mat, -b_vec)
    if sol.success == True:
        return True
    else:
        return False


def readmiplib(fname):
    """
    Parses a MIPLIB (.mps) file to obtain A, b in inequality
    form:
    
        { x | A x >= b }
    
    Data is then saved in a pickle.
    """
    row_dict = {}  # row name : sign
    col_tups = []
    bound_tups = []
    rhs_dict = {}  # row name : b vector
    with open(fname, 'r') as file:
        line = next(file)
        position = None
        while line:
            # determine position
            if 'ROWS' in line:
                position = 'ROWS'
            if 'COLUMNS' in line:
                position = 'COLUMNS'
            if 'RHS' in line:
                position = 'RHS'
            if 'BOUNDS' in line:
                position = 'BOUNDS'
            if 'ENDATA' in line:
                break 
            parsed = line.split()
        
            if position == 'ROWS':
                if 'ROWS' in parsed[0]:
                    pass
                else:
                    row_dict[parsed[1]] = parsed[0]
                    
            if position == 'COLUMNS':
                if 'COLUMNS' in parsed[0] or 'MARK' in parsed[0]:
                    pass
                else:
                    if len(parsed) == 3:
                        col_tups.append(parsed)
                    else:
                        col_tups.append(( parsed[0], parsed[1], parsed[2] ))
                        col_tups.append(( parsed[0], parsed[3], parsed[4] ))
            
            if position == 'RHS':
                if len(parsed) == 1:
                    pass
                else:
                    if len(parsed) == 3:
                        rhs_dict[parsed[1]] = parsed[2]
                    else:
                        rhs_dict[parsed[1]] = parsed[2]
                        rhs_dict[parsed[3]] = parsed[4]
            
            if position == 'BOUNDS':
                if 'BOUNDS' in parsed[0]:
                    pass
                else:
                    bound_tups.append(parsed)
                    if 'LI' in parsed[0] or 'LB' in parsed[0] or 'LO' in parsed[0]:
                        row_dict[parsed[0] + parsed[2]] = 'G'
                        rhs_dict[parsed[0] + parsed[2]] = parsed[3]
                    elif 'UI' in parsed[0] or 'UB' in parsed[0] or 'UP' in parsed[0]:
                        row_dict[parsed[0] + parsed[2]] = 'L'
                        rhs_dict[parsed[0] + parsed[2]] = parsed[3]
                    elif 'EQ' in parsed[0] or 'FX' in parsed[0]:
                        row_dict[parsed[0] + parsed[2]] = 'E'
                        rhs_dict[parsed[0] + parsed[2]] = parsed[3]
                    elif 'PL' in parsed[0]:
                        row_dict[parsed[0] + parsed[2]] = 'G'
                        rhs_dict[parsed[0] + parsed[2]] = 0
                    elif 'FR' in parsed[0]:
                        bound_tups.pop()
                    else:
                        print('Error parsing bound {}'.format(parsed[2]))
                        # exit()
   
            line = next(file, None)

    # Go through rows, bounds and swap all equality with two inequalities
    rows_to_remove = []
    for row in row_dict:
        if row_dict[row] == 'E':
            rows_to_remove.append(row)
            row_ub = row + 'UB'
            row_lb = row + 'LB'
            cols_to_add = []
            for col in col_tups:
                if col[1] == row:
                    cols_to_add.append(( col[0], row_ub, col[2] ))
                    cols_to_add.append(( col[0], row_lb, col[2] ))
                    col_tups.remove(col)
            col_tups += cols_to_add
    
    for row in rows_to_remove:
        row_ub = row + 'UB'
        row_lb = row + 'LB'
        row_dict[row_ub] = 'G'
        row_dict[row_lb] = 'L'
        rhs_dict[row_ub] = rhs_dict.get(row, 0)
        rhs_dict[row_lb] = rhs_dict.get(row, 0)
        row_dict.pop(row, 0)
        rhs_dict.pop(row, 0)

    # generate dictionary form
    row_names = sorted(list(row_dict.keys()))
    col_names = [ ix[0] for ix in col_tups ]
    col_names = sorted(list(set(col_names)))
    A_mat_dict = {}
    obj_row = ''
    for row in row_names:
        if row_dict[row] == 'N':
            obj_row = row
        else:
            A_mat_dict[row] = { col:0 for col in col_names }
    row_names.remove(obj_row)
    for col in col_tups:
        if col[1] in row_names:
            A_mat_dict[col[1]][col[0]] = col[2]
    for bound in bound_tups:
        A_mat_dict[bound[0] + bound[2]][bound[2]] = 1
    
    # generate A, b matrices
    m, n = len(row_names), len(col_names)
    A_mat = np.zeros((m, n))
    b_vec = np.zeros(m)
    for ix in range(m):
        row = row_names[ix]
        if row_dict[row] == 'G':
            mult = 1
        elif row_dict[row] == 'L':
            mult = -1
        else:
            print('Error generating A. Did not recognize sign {}'.format(row_dict[row]))
        
        for iy in range(n):
            col = col_names[iy]
            A_mat[ix, iy] = mult * float(A_mat_dict[row][col])
        
        b_vec[ix] = float(rhs_dict.get(row, 0))
    
    return A_mat, b_vec

fname = 'gen-ip054.mps'
# fname = 'miplib/markshare_4_0.mps'
readmiplib(fname)



(array([[ 1.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  1.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  1.        , ...,  0.        ,
          0.        ,  0.        ],
        ...,
        [ 0.3972443 , -0.14191287, -0.35597819, ...,  0.7222276 ,
          0.2602271 , -0.34660471],
        [ 1.16352542, -0.        , -0.86536518, ...,  0.16431961,
         -0.        , -0.        ],
        [-0.        , -0.        , -0.        , ..., -0.        ,
         -0.        , -0.        ]]),
 array([   0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.        ,    0.        ,    0.        ,
           0.        ,    0.     

In [ ]:
from pathlib import Path
import pickle

# params
make_relaxations = True
n_relaxations = 1
p = Path('miplib/')

for fname in p.iterdir():
    if fname.suffix == '.mps':
        print(fname)
        A_mat, b_vec = readmiplib(fname)
        feas_exists = check_lpsolver(A_mat, b_vec)
        if feas_exists is True:
            print('... is valid.')
        else:
            print('... is not valid.')
        # normalize the rows
        m, n = A_mat.shape
        for ix in range(m):
            A_mat[ix] = A_mat[ix] / np.linalg.norm(A_mat[ix])
            b_vec[ix] = b_vec[ix] / np.linalg.norm(A_mat[ix])
        # generate relaxations
        if make_relaxations is True:
            for ix in range(n_relaxations):
                b_relax = b_vec - np.random.exponential(scale=1, size=m)
                feas_set = { 'A_mat': A_mat, 'b_vec': b_vec, 'b_relax': b_relax }
                fname_pickle = p / (fname.stem + '_R{}.pickle'.format(ix))
                if feas_exists:
                    with open(fname_pickle, 'wb') as f:
                        pickle.dump(feas_set, f)
            pass
        else:
            feas_set = { 'A_mat': A_mat, 'b_vec': b_vec }
            fname_pickle = p / (fname.stem + '.pickle')
            if feas_exists:
                with open(fname_pickle, 'wb') as f:
                    pickle.dump(feas_set, f)
